In [380]:
from PIL import Image
from scipy import ndimage
import copy
import csv
import json
import numpy as np
import math
import matplotlib.pyplot as plt
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2 # import after setting OPENCV_IO_MAX_IMAGE_PIXELS
from datetime import datetime

In [381]:
def get_angle_simple(a, b):
    
    #Calculates the angle of a straight line (with respect to the horizon) between points a and b:
    return math.degrees(math.atan2(a[1] - b[1], a[0] - b[0]))

In [382]:
def get_angle_simple_vert(a, b):
    
    #Calculates the angle of a straight line (with respect to the horizon) between points a and b:
    return math.degrees(math.atan2(b[0] - a[0], a[1] - b[1]))

In [383]:
def get_angle_averaged(alignment_pts):
    
    #For a set of points a, b, c, d in alignment_pts, calculated the angle of each straight line edge of the quadrilateral (ie. get the angle of A-B, A-C, B-D, C-D) and average them:
    angle1 = get_angle_simple(alignment_pts[1], alignment_pts[0])
    angle2 = get_angle_simple(alignment_pts[3], alignment_pts[2])
    angle3 = get_angle_simple_vert(alignment_pts[2], alignment_pts[0])
    angle4 = get_angle_simple_vert(alignment_pts[3], alignment_pts[1])
    
    angle = (angle1 + angle2 + angle3 + angle4)/4
    return angle

In [384]:
def crop_rotated(img, h_old, w_old, angle):
    
    #Rotates an image by a given angle... then crops the image to remove the rotational defects:
    a = abs(math.radians(angle))
    sin_a = abs(math.sin(a))
    h, w = img.shape[0], img.shape[1]
    dh, dw = h_old * sin_a, w_old * sin_a
    hc, wc = h - 2 * dh, w - 2 * dw
    
    return img[int((h-hc)/2):int((h+hc)/2), int((w-wc)/2):int((w+wc)/2)]

In [385]:
def make_label(pts, img, theta, ox, oy):
    
    #Makes labels in the image for die offsets and rotation:
    data_pts = np.empty((len(pts), 2), dtype=np.float64)
    
    for i in range(data_pts.shape[0]):
        
        data_pts[i, 0] = pts[i, 0, 0]
        data_pts[i, 1] = pts[i, 0, 1]
        mean = np.empty((0))
        mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
        cntr = (int(mean[0, 0]), int(mean[0, 1]))
        label_rotation = " Rotation: " + ("%.4f" % theta) + " degrees"
        label_offset = " Offset: " + ("(%.4f, %.4f)" % (ox, oy) + " um")
        textbox = cv2.rectangle(img, (cntr[0], cntr[1] - 25), (cntr[0] + 300, cntr[1] + 45), (255, 255, 255), -1)
        cv2.putText(img, label_rotation, (cntr[0], cntr[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(img, label_offset, (cntr[0], cntr[1] + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

In [386]:
def order_alignment_marker_list(alignment_pts):
    
    #Orders the list of global alignment marker positions such that it is in the order LL, LR, UL, UR:
    new_list = alignment_pts.copy()
    sum = np.zeros((np.shape(alignment_pts)[0]), dtype='int')
    h = 0
    
    while h < 2:
        
        i = 2*h
        j = i + 1
        sum[i] = new_list[i][0] + new_list[i][1]
        sum[j] = new_list[j][0] + new_list[j][1]
        
        if sum[j] < sum[i]:
            
            new_list[i] = alignment_pts[j]
            new_list[j] = alignment_pts[i]
        
        h = h + 1
    
    alignment_pts = new_list.copy()
    
    return alignment_pts

In [387]:
def calculate_scale_using_alignment_markers(alignment_pts, alignment_mark_dist):
    
    #Image scale is calculed using the global alignment marker coordinates and given marker to marker distance:
    d01 = np.absolute(alignment_pts[0] - alignment_pts[1])
    d02 = np.absolute(alignment_pts[0] - alignment_pts[2])
    d13 = np.absolute(alignment_pts[1] - alignment_pts[3])
    d23 = np.absolute(alignment_pts[2] - alignment_pts[3])
    
    dist01 = np.linalg.norm(d01)
    dist02 = np.linalg.norm(d02)
    dist13 = np.linalg.norm(d13)
    dist23 = np.linalg.norm(d23)
    
    max_vertical_dist = max(d01[0], d02[0], d13[0], d23[0])
    max_horizontal_dist = max(d01[1], d02[1], d13[1], d23[1])
    rescale_amount = max_horizontal_dist / max_vertical_dist
    avg_pixel_dist = (dist01 + dist02 + dist13 + dist23) / 4
    pixel_um_scale = alignment_mark_dist / avg_pixel_dist
    
    print('Pixel to um scale (w.r.t. alignment markers) [um/pixel]:', pixel_um_scale)
    
    return pixel_um_scale, rescale_amount

In [388]:
def match_template(result, img, template, template_name, threshold):
    
    #Match template based on threshold value:
    w, h = template.shape[::-1]
    
    #Using matching operation TM_CCORR_NORMED (23/01/30):
    res = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    
    n = np.count_nonzero(res >= threshold)
    pts = np.dstack(np.unravel_index(np.argsort(res.ravel()), res.shape))[0]
    pts = pts[::-1][:n]
    pts[:, [1, 0]] = pts[:, [0, 1]]
    
    delete = []
    
    for i in range(len(pts)):
        
        for j in range(i + 1, len(pts)):
            
            if np.linalg.norm(pts[i] - pts[j]) <= 15 and j not in delete:
                
                delete.append(j)
                
    pts = np.delete(pts, delete, axis=0)
    pts = pts[np.lexsort((pts[:, 0], pts[:, 1]))]
    
    for pt in pts:
        
        cv2.rectangle(result, (pt[0], pt[1]), (pt[0] + w, pt[1] + h), (0, 0, 255), 8)
        
    print("Detected " + template_name + " markers:", len(pts))
    
    return pts, result

In [389]:
def identify_dies(contours, alignment_pts, min_die_area=20000, max_die_area=100000):
    
    #Identify dies with thresholding/contouring based on a minimum and maximum die area:
    nontrivial_contours = []
    nontrivial_offsets = []
    
    for i, c in enumerate(contours):
        
        area = cv2.contourArea(c)
        #print(area)
        
        if area < min_die_area or area > max_die_area:
            
            continue
            
        x, y, w, h = cv2.boundingRect(c)
        
        is_alignment_mark = False
        
        for alignment_pt in alignment_pts:
            
            if math.dist([x, y], alignment_pt) < 300:
                
                is_alignment_mark = True
                
                break
                
        if is_alignment_mark:
            
            continue
        
        nontrivial_contours.append(c)
        nontrivial_offsets.append([x, y])
    
        
    nontrivial_contours.sort(key=lambda c: (cv2.boundingRect(c)[0], cv2.boundingRect(c)[1]))
    nontrivial_offsets.sort(key=lambda o: (o[0], o[1]))
    x_coords = [o[0] for o in nontrivial_offsets]
    y_coords = [o[1] for o in nontrivial_offsets]

    # Use np.lexsort with the tuple of arrays
    sorted_indices = np.lexsort((y_coords, x_coords))

    # Use sorted indices to sort nontrivial_offsets
    sorted_nontrivial_offsets = np.array(nontrivial_offsets)[sorted_indices]

    nontrivial_offsets = np.array(nontrivial_offsets)
    
    return nontrivial_contours, nontrivial_offsets

In [390]:
def identify_corner_markers(result, w1, h1, w2, h2, template1_pts, template2_pts):
    
    #Identify the corner marker locations with a die based on template matching and draw box:
    T1 = np.array(template1_pts)
    T2 = np.array(template2_pts)
    c1 = np.array([w1, h1])
    c2 = np.array([w2, h2])
    
    if len(T1) < len(T2):
        lengthT = len(T1)
    else:
        lengthT = len(T2)
    
    i = 0
    while i < lengthT - 1:
        box = np.int0([T1[i] + c1, T1[i+1] + c1, T2[i+1] + c2, T2[i] + c2])
        #cv2.polylines(result, [box], True, (255, i * 30 % 255, 0), 3)
        
        #p = [box[0][0], box[0][1] + 100]
        #theta = get_angle(box[1], box[0], p)
        #theta = 0 - get_angle_simple(box[1], box[0])
        #theta = min(abs(theta), 90 - abs(theta))
        #cntr = [box[0][0], box[0][1]]
        # textbox = cv2.rectangle(result, (cntr[0], cntr[1] - 25), (cntr[0] + 250, cntr[1] + 20), (255, 255, 255), -1)
        # label_rotation = " Rotation: " + ("%.4f" % theta) + " degrees"
        # cv2.putText(result, label_rotation, (cntr[0], cntr[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        i += 2
    
    return result

In [391]:
def calculate_offset(alignment_pts, die_offsets, ideal_param, image_paths):
    # calculates the distance between the UL alignment marker and the UL die contour
    result = cv2.imread(image_paths['input'])
    print("UL ALIGNMENT PT", alignment_pts[2])
    x,y = alignment_pts[0]
    cv2.circle(result, (x, y), 10, (0, 92, 255), -1)

    dx, dy = die_offsets[0][0], die_offsets[0][1] 
    cv2.circle(result, (int(dx), int(dy)), 10, (100, 92, 255), -1)
    dx 
    dy 
    print(dx)
    print(dy)
    cv2.circle(result, (int(dx), int(dy)), 10, (0, 92, 255), -1)
    x_offset = x-dx
    y_offset = -(y-dy)
    print("x offset", x_offset)
    print("y offset", y_offset)
    return x_offset, y_offset, result

    #find the distance between the alignment+pt and the first die_offset by subtracting x and y coord
    
    

In [392]:
def identify_misalignment(result, die_contours, die_offsets, alignment_pts, ideal_param, image_paths):
    
    
    #Identify misalignment of dies with respect to the expected location given a known shift in the assembly during die placement:
    # midpoint according to alignment markers and assembly offset
    #ideal_param['x_midpoint'] = x_midpoint
    #ideal_param['y_midpoint'] = y_midpoint
    scale = ideal_param['scale']
    ideal_param['x_midpoint'] = np.mean(alignment_pts[:, 0]) + ideal_param['assembly_x_offset'] / scale
    ideal_param['y_midpoint'] = np.mean(alignment_pts[:, 1]) - ideal_param['assembly_y_offset'] / scale
    x_midpoint = ideal_param['x_midpoint']
    y_midpoint = ideal_param['y_midpoint']
    num_rows = ideal_param['num_rows']
    num_cols = ideal_param['num_cols']
    # die width and pitch in pixels
    die_width = 1.1*ideal_param['die_width'] / scale
    pitch = ideal_param['pitch'] / scale
    # ideal top left corner of die layout
    dies_layout_vertical_width = (num_rows - 1) * pitch + die_width
    dies_layout_horizontal_width = (num_cols - 1) * pitch + die_width
    x_start = x_midpoint - dies_layout_vertical_width // 2
    y_start = y_midpoint - dies_layout_horizontal_width // 2
    ideal_param['x_start'] = x_start
    ideal_param['y_start'] = y_start
    ideal_pts = np.zeros((num_rows, num_cols, 2))
    
    for r in range(num_rows):
        
        for c in range(num_cols):
            
            ideal_pts[r, c, 0] = x_start + r * pitch
            ideal_pts[r, c, 1] = y_start + c * pitch
            tl = np.int0([ideal_pts[r, c, 0], ideal_pts[r, c, 1]])
            br = np.int0([ideal_pts[r, c, 0] + die_width, ideal_pts[r, c, 1] + die_width])
            cv2.rectangle(result, tl, br, (0, 0, 255), 5)
    
    shifts_relative = {}
    shifts_absolute = {}
    midpts = np.zeros((num_rows, num_cols, 2))
    
    for i, contour in enumerate(die_contours):
        
        x, y, w, h = cv2.boundingRect(contour)
        
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(result, [box], 0, (0, 255, 0), 8)
        
        theta = rect[2]
        sign_theta = np.sign(theta)
        
        if abs(theta) == 0 or abs(theta) == 90:
            
            theta = 0
        
        theta = sign_theta * min(abs(theta), 90 - abs(theta))
        p = np.array([x, y])
        dists = np.sum((ideal_pts - p)**2, axis=-1)
        idx = np.unravel_index(np.argmin(dists), dists.shape)
        r, c = int(idx[0]), int(idx[1])
        midpts[r, c, 0] = x + w / 2
        midpts[r, c, 1] = y + h / 2
        ox = scale * (x - ideal_pts[r, c, 0])
        oy = - scale * (y - ideal_pts[r, c, 1])
        ox_absolute = scale * (x - x_start)
        oy_absolute = - scale * (y - y_start)
        
        if r not in shifts_relative:
            
            shifts_relative[r] = {}
            shifts_absolute[r] = {}
        
        shifts_relative[r][c] = {'x': ox, 'y': oy, 'theta': theta}
        shifts_absolute[r][c] = {'x': ox_absolute, 'y': oy_absolute, 'theta': theta}
        #make_label(contour, result, theta, ox, oy)
    
    ideal_param['midpts'] = midpts
   
    
    for i, c in enumerate(alignment_pts):
        
        ox = scale * (c[0] - x_start)
        oy = - scale * (c[1] - y_start)
        shifts_absolute['a' + str(i)] = {'x': ox, 'y': oy}
    
    # TODO: rewrite / confirm !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
    with open(image_paths['scan_dir'] + '/shifts_relative.json', 'w') as f:
        json.dump(shifts_relative, f)
    with open(image_paths['scan_dir'] + '/shifts_absolute.json', 'w') as f:
        json.dump(shifts_absolute, f)
    
    return result

In [393]:
def identify_misalignment_with_corner_markers(template1_pts, template2_pts, alignment_pts, ideal_param, image_paths):
    
    #Identify die misalignment with corner markers and export to CSV:
    scale = ideal_param['scale']
    midpts = ideal_param['midpts']#expected midpt of unshifted die, 2d matrix that is 5x5
    x_midpoint = ideal_param['x_midpoint']
    y_midpoint = ideal_param['y_midpoint']
    
    max_dist_to_midpt = ((ideal_param['die_width']/2)**2 + (ideal_param['die_height']/2)**2)/(scale**2)
    oc = ideal_param['oc']
    
    dies_to_markers = {}
    shifts = {}
    shifts_csv = {}
    
    for p in template1_pts:
        
        dists = np.sum((midpts - p)**2, axis=-1)
        
        
        # find the index (row, column) of the closest corresponding die
        idx = np.unravel_index(np.argmin(dists), dists.shape)#converts index back to 2d shape of dists, which is formed by shape of midpts
        
        if dists[idx] > max_dist_to_midpt:
            
            continue
        
        r, c = int(idx[0]), int(idx[1])
        print(f"dies to markers: {r},{c}")
        
        # account for defects
        if r not in dies_to_markers:
            
            dies_to_markers[r] = {}
        
        if c not in dies_to_markers[r]:
            
            dies_to_markers[r][c] = []
        
        dies_to_markers[r][c].append(p)#NEED TO ADD .TOLIST AFTER TESTING
        
        if len(dies_to_markers[r][c]) == 2:
            
            markers = dies_to_markers[r][c]
            markers.sort(key=lambda m: (m[0], m[1]))
            theta = get_angle_simple(markers[1], markers[0])
            sign_theta = - np.sign(theta)
            theta = sign_theta * min(abs(theta), 90 - abs(theta))
            
            
            if r not in shifts:
                
                shifts[r] = {}
            
            if c not in shifts[r]:
                print(f"shifts csv: {r},{c}")
                shifts[r][c] = {}
            
            shifts[r][c]['theta'] = theta
            shifts_csv[str(r) + ', ' + str(c)] = {'theta': theta}
        
    for p in template2_pts:
        
        dists = np.sum((midpts - p)**2, axis=-1)
        idx = np.unravel_index(np.argmin(dists), dists.shape)
        
        if dists[idx] > max_dist_to_midpt:
            
            continue
        
        r, c = int(idx[0]), int(idx[1])
        
        if r not in dies_to_markers:
            
            dies_to_markers[r] = {}
        
        if c not in dies_to_markers[r]:
            
            dies_to_markers[r][c] = []
        
        dies_to_markers[r][c].append(p)#NEED TO ADD .TOLIST AFTER TESTING
    
    for r in shifts:
        
        for c in shifts[r]:
            
            markers = dies_to_markers[r][c]
            # skip defects (TODO: fixable?)
            
            if len(markers) < 4:
                
                continue
            
            # compute average of four corner markers
            die_center = np.mean(markers, axis=0) # + oc
            ox = scale * (die_center[0] - x_midpoint)
            oy = - scale * (die_center[1] - y_midpoint)
            shifts[r][c]['x'] = ox
            shifts[r][c]['y'] = oy
            shifts_csv[str(r) + ', ' + str(c)]['x'] = ox
            shifts_csv[str(r) + ', ' + str(c)]['y'] = oy
    
    for i, c in enumerate(alignment_pts):
        
        ox = scale * (c[0] - x_midpoint)
        oy = - scale * (c[1] - y_midpoint)
        shifts['alignment ' + str(i)] = {'x': ox, 'y': oy, 'theta': 0}
        shifts_csv['alignment ' + str(i)] = {'x': ox, 'y': oy, 'theta': 0}
    
    with open(image_paths['scan_dir'] + '/shifts.json', 'w') as f:
        
        json.dump(shifts, f)
    
    with open(image_paths['scan_dir'] + '/shifts.csv', 'w') as f:
        
        fields = ['pos', 'x', 'y', 'theta']
        w = csv.DictWriter(f, fields)
        w.writeheader()
        
        for k, v in sorted(shifts_csv.items()):
            
            row = {'pos': k}
            row.update(v)
            w.writerow(row)
    return shifts, shifts_csv

In [394]:
def method_2(shifts, shifts_csv, die_offsets, die_contours, alignment_pts, image_paths, ideal_param):
    #loop through the angle rotation of the dies and rotate and crop entire image
    cross = cv2.imread(image_paths['cross'])
    squares = cv2.imread(image_paths['squares'])
    out = cv2.imread(image_paths['input'])
    decoy = out.copy()
    h,w = cross.shape[1], cross.shape[0]
    scale = ideal_param['scale']
    midpts = ideal_param['midpts']
    x_midpoint = ideal_param['x_midpoint']
    y_midpoint = ideal_param['y_midpoint']
    
    
    i = 0
    
    
    for r in range(ideal_param['num_rows']):#TODO: might need to make this dependent on keys of shifts
        for c in range(ideal_param['num_cols']):
            
            
            theta = shifts[r][c]['theta']
            #rotate/crop template, use previously found die contours to cut die out of entire image, template match all 4 corner markers, find centers of all 4 corners
            rot_cross = crop_rotated(cross, h, w, theta)
            rot_squares = crop_rotated(squares, h, w, theta)
            rot_cross = cv2.cvtColor(rot_cross, cv2.COLOR_BGR2GRAY)
            rot_squares = cv2.cvtColor(rot_squares, cv2.COLOR_BGR2GRAY)
            out.astype(np.uint8)
            rot_squares.astype(np.uint8)
            rot_cross.astype(np.uint8)
            
            w, h = rot_cross.shape[0], rot_cross.shape[1]
            
            _, _, die_w, die_h = cv2.boundingRect(die_contours[i])
            roi = out[die_offsets[i][1]:die_offsets[i][1]+ die_h, die_offsets[i][0]:die_offsets[i][0]+ die_w]
            roi.astype(np.uint8)
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

            #perform template matching on die
            cross_pts, _ = match_template(decoy, roi, rot_cross, f"rotated cross [{r}][{c}]", .7)
            squares_pts, _ = match_template(decoy, roi, rot_squares, f"rotated squares [{r}][{c}]", .8)

            x, y = die_offsets[i][0], die_offsets[i][1]
            # Draw a rectangle on the original image of die with no rotation
            cv2.rectangle(out, (x, y), (x+die_w, y+die_h), (0, 255, 0), 4)  # Green rectangle, thickness 2
            cv2.putText(out, f"r:{r} c:{c}, i:{i} die offset", (x - 25, y - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (240, 32, 160), 4)

            #add offset from die cutout to template matched coords to get actual coords according to coord system fo entire image
            offset_x, offset_y = die_offsets[i]
            offset = np.array([[offset_x, offset_y]])
            cross_pts += offset
            squares_pts += offset
            
            for x,y in cross_pts:
                cv2.circle(out, (x, y), 5, (0, 0, 255), -1)
                cv2.putText(out, "corner", (x - 25, y - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.rectangle(out, (x, y), (x + w, y + h), (0, 0, 255), 5)
            
        
            for x,y in squares_pts:
                cv2.circle(out, (x, y), 5, (0, 0, 255), -1)
                cv2.putText(out, "corner", (x - 25, y - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.rectangle(out, (x, y), (x + w, y + h), (0, 0, 255), 5)

            #calculate centroid by adding half the width and subtracting half the height of rot_template
            center_offset = np.array([[w//2, h//2]]) #why do we add height instead of subtract?
            center_offset = np.repeat(center_offset, len(cross_pts), axis=0)#ensures shapes match for broadcasting adding
            cross_pts += center_offset

            center_offset2 = np.array([[w//2, h//2]]) #why do we add height instead of subtract? +y goes down 
            center_offset2 = np.repeat(center_offset2, len(squares_pts), axis=0)#ensures shapes match for broadcasting adding
            print("cross_pts centers, should be 2", len(cross_pts))
            print("square_pts centers, should be 2", len(squares_pts))
            squares_pts += center_offset2


            #draw centroids onto image
            for x, y in cross_pts:
                cv2.circle(out, (x, y), 5, (255, 0, 0), -1)
                cv2.putText(out, "centroid", (x - 25, y - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            for x, y in squares_pts:
                cv2.circle(out, (x, y), 5, (255, 0, 0), -1)
                cv2.putText(out, "centroid", (x - 25, y - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            #calculate and draw center of die
            cross_pts = np.array(cross_pts)
            squares_pts = np.array(squares_pts)
            pts = np.vstack((cross_pts, squares_pts))
            die_center = np.mean(pts, axis=0)
            ox = die_center[0]
            oy = die_center[1] #shouldnt have a negative scale
            shifts[r][c]['x'] =  ox
            shifts[r][c]['y'] =  oy
            shifts_csv[str(r) + ', ' + str(c)]['x'] =scale * ox
            shifts_csv[str(r) + ', ' + str(c)]['y'] = - scale * oy
            if r == 4 and c == 2:
                cv2.circle(out, (int(ox), int(oy)), 5, (0, 255, 255), -1)#yellow = 4,2
                cv2.putText(out, "4,2 center of die", (int(ox) - 25, int(oy) - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            elif r==4 and c==3:
                cv2.circle(out, (int(ox), int(oy)), 5, (180, 105, 255), -1)#pink = 4,3
                cv2.putText(out, "4,3 center of die", (int(ox) - 25, int(oy) - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                cv2.circle(out, (int(ox), int(oy)), 5, (0, 255, 0), -1)
                cv2.putText(out, f"{r},{c} center of die", (int(ox) - 25, int(oy) - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            i += 1

   
            
    for i, p in enumerate(alignment_pts):
            shifts['alignment ' + str(i)] = {'x': p[0], 'y': p[1], 'theta': 0}
            shifts_csv['alignment ' + str(i)] = {'x': scale * p[0], 'y': -scale * p[1], 'theta': 0}
            

            
    
    #sets up csv file
    with open(image_paths['scan_dir'] + '/method2_shifts_final.csv', 'w') as f:
        fields = ['pos', 'x', 'y', 'theta']
        w = csv.DictWriter(f, fields)
        w.writeheader()
        #key,value
        for k, v in sorted(shifts_csv.items()):
            row = {'pos': k}
            row.update(v)
            w.writerow(row)
    
            


            


# ANALYSIS

In [395]:
def analysis(image_paths, ideal_param, scale_to_use='detected', needs_rotation=False, has_defects=False):
    
    #Read image and convert to grayscale:
    img = cv2.imread(image_paths['input'])
    result = img.copy()
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #Identify global alignment markers, adjust rotation, and rescale dimensions:
    template_alignment = cv2.imread(image_paths['alignment'])
    template_alignment_gray = cv2.cvtColor(template_alignment, cv2.COLOR_BGR2GRAY)
    alignment_pts, result = match_template(result, img_gray, template_alignment_gray, "alignment", 0.9)
    alignment_pts = order_alignment_marker_list(alignment_pts)
    ideal_param['scale'], rescale = calculate_scale_using_alignment_markers(alignment_pts, ideal_param['alignment_mark_dist'])
    ideal_param['min_die_area'] = 165000
    ideal_param['max_die_area'] = 650000
    img = cv2.resize(img, (int(rescale*img.shape[1]), img.shape[0]), interpolation=cv2.INTER_CUBIC)
    result = img.copy()
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    theta = 0 - get_angle_averaged(alignment_pts)
    print("Wafer scan rotation [degrees]:", theta)
    
    #Use given scale instead of calculated scale:
    if scale_to_use == 'given':
        
        ideal_param['scale'] = pixel_to_um_scale_given[i]
    
    #If the image needs rotation, rotate and crop the image, then re-identify global alignment markers and re-calculated scale:
    if needs_rotation:
        
        i = 0
        #for i in range(ideal_param['number_of_rotation_correction']):
        while abs(theta) > ideal_param['target_rotation']:

            if i == int(ideal_param['number_of_rotation_correction']):
                break
            
            h, w = img.shape[0], img.shape[1]
            img = ndimage.rotate(img, -theta)
            img = crop_rotated(img, h, w, -theta)
            result = img.copy()
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            alignment_pts, result = match_template(result, img_gray, template_alignment_gray, "alignment", 0.9)
            alignment_pts = order_alignment_marker_list(alignment_pts)
            ideal_param['scale'], _ = calculate_scale_using_alignment_markers(alignment_pts, ideal_param['alignment_mark_dist'])
            ideal_param['min_die_area'] = (0.9*ideal_param['die_width']/ideal_param['scale'])*(0.9*ideal_param['die_height']/ideal_param['scale'])
            ideal_param['max_die_area'] = (1.1*ideal_param['die_width']/ideal_param['scale'])*(1.1*ideal_param['die_height']/ideal_param['scale'])
            theta = 0 - get_angle_averaged(alignment_pts)
            print("Wafer scan rotation [degrees]:", theta)
            i += 1

    #Calculates midpoint of the global alignemnt markers:
    w0, h0 = template_alignment_gray.shape[::-1]
    
    for i, c in enumerate(alignment_pts):
        
        alignment_pts[i] = np.array([c[0] + w0 // 2, c[1] + h0 // 2])
    
    #Use CV2 thresholding to get all contours in the image:
    _, bw = cv2.threshold(img_gray, 232, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    cv2.imwrite('./input_images/threshold.png', bw)
    
    contours, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    #Identify dies by searching for contours with area matching the expected die area such that min area = 0.81*die area and max area = 1.21*die area:
    die_contours, die_offsets = identify_dies(contours, alignment_pts, min_die_area=ideal_param['min_die_area'], max_die_area=ideal_param['max_die_area'])

    x_off, y_off, result = calculate_offset(alignment_pts, die_offsets, ideal_param, image_paths) 
    ideal_param['assembly_x_offset'] = -1600
    ideal_param['assembly_y_offset'] = 2030

    result = identify_misalignment(result, die_contours, die_offsets, alignment_pts, ideal_param, image_paths)
    cv2.imwrite("/Users/travisha/Downloads/CHIPS_research/identifymisalign.jpg", result)

   
    
    # #Match templates of the die alignment markers (crosses/squares):
    # template_cross = cv2.imread(image_paths['cross'])
    # template_squares = cv2.imread(image_paths['squares'])
    # template_cross_gray = cv2.cvtColor(template_cross, cv2.COLOR_BGR2GRAY)
    # template_squares_gray = cv2.cvtColor(template_squares, cv2.COLOR_BGR2GRAY)

    # cross_thres = .77
    # square_thres = .789
    # cross_pts, result = match_template(result, img_gray, template_cross_gray, "cross", cross_thres)
    # squares_pts, result = match_template(result, img_gray, template_squares_gray, "squares", square_thres)
    # numcross = len(cross_pts)
    # numsqaures = len(squares_pts)
    # while  numcross!= 50:
    #     if(numcross < 50):
    #         cross_thres -= .01
    #     if(numcross > 50):
    #         cross_thres += .01
    #     print("cross thresh:", cross_thres)
    #     cross_pts, result = match_template(result, img_gray, template_cross_gray, "cross", cross_thres)
    #     numcross = len(cross_pts)
    # rep = 5
    # while  numsqaures!= 50 and rep > 0:
    #     if numsqaures < 50:
    #         square_thres -= .002
    #     if numsqaures > 50:
    #         square_thres += .001
    #     print("square thres:", square_thres)
    #     squares_pts, result = match_template(result, img_gray, template_squares_gray, "squares", square_thres)
    #     numsqaures = len(squares_pts)
    #     rep-=1
    

    
    # #Identify the alignment markers belonging to each die and calculate die offsets based on those alignment marks:
    # w1, h1 = template_cross_gray.shape[::-1]
    # w2, h2 = template_squares_gray.shape[::-1]
    # result = identify_corner_markers(result, w1//2, h1//2, w2//2, h2//2, cross_pts, squares_pts)#correctly detects all dies
    # ideal_param['oc'] = [(w1 + w2) / 2, (h1 + h2) / 2]
    # shifts, shifts_csv = identify_misalignment_with_corner_markers(cross_pts, squares_pts, alignment_pts, ideal_param, image_paths)
    # method_2(shifts, shifts_csv, die_offsets, die_contours, alignment_pts, image_paths, ideal_param)
    
    # #Create output bitmap:
    # cv2.imwrite(image_paths['output'], result)

In [396]:
# TODO: calculate pitch using midpoint / avg
# average distance between top left corners of two adjacent dies: ~402 pixels <==> 1634 um
# average h, w: ~246 pixels <==> 1000 um
# average spacing between dies: ~156 pixels <==> 634 um
scan_dir = '/Users/travisha/Downloads/CHIPS_research'
image_paths = {}
image_paths['scan_dir'] = scan_dir
image_paths['input'] = scan_dir + '/v1_Sample2_Scan2_corrected.png'
image_paths['output'] = scan_dir + '/output.bmp'
image_paths['corners'] = scan_dir + '/cornersmethod2.png'
image_paths['alignment'] = scan_dir + '/alignment_quarter.bmp'
image_paths['cross'] = scan_dir + '/cross_quarter.bmp'
image_paths['squares'] = scan_dir + '/squares_quarter.bmp'
image_paths['cross_blurred'] = None
image_paths['squares_blurred'] = None
pixel_to_um_scale_given = None
pixel_to_um_scale_detected = None
ideal_param = {}
ideal_param['num_rows'] = 5
ideal_param['num_cols'] = 5
ideal_param['die_width'] = 2000 #[um]
ideal_param['die_height'] = 2000 #[um]
ideal_param['pitch'] = 2800 #[um]
ideal_param['alignment_mark_dist'] = 25000 #[um]

ideal_param['number_of_rotation_correction'] = 1
ideal_param['target_rotation'] = 0.0015 #[degrees]

start_time = datetime.now()
print("Start Time =", start_time)

analysis(image_paths, ideal_param, scale_to_use='detected', needs_rotation=True, has_defects=False)

end_time = datetime.now()
print("End Time =", end_time)
run_time = end_time - start_time
print("Run Time =", run_time)
# TODO: switched rows with columns?

Start Time = 2024-09-13 12:45:23.052797
Detected alignment markers: 4
Pixel to um scale (w.r.t. alignment markers) [um/pixel]: 2.207990242824598
Wafer scan rotation [degrees]: 0.08603387639409091
Detected alignment markers: 4
Pixel to um scale (w.r.t. alignment markers) [um/pixel]: 2.2070668458626375
Wafer scan rotation [degrees]: 0.0012631316489668983
UL ALIGNMENT PT [  615 11612]
2583
2056
x offset -1964
y offset 1768


/var/folders/vs/q5nlgtbd0h928z1tq9v9s8440000gq/T/ipykernel_43269/3857381294.py:33: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  tl = np.int0([ideal_pts[r, c, 0], ideal_pts[r, c, 1]])
/var/folders/vs/q5nlgtbd0h928z1tq9v9s8440000gq/T/ipykernel_43269/3857381294.py:34: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  br = np.int0([ideal_pts[r, c, 0] + die_width, ideal_pts[r, c, 1] + die_width])
/var/folders/vs/q5nlgtbd0h928z1tq9v9s8440000gq/T/ipykernel_43269/3857381294.py:47: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


End Time = 2024-09-13 12:47:22.558934
Run Time = 0:01:59.506137


In [397]:
# from scipy.optimize import root
# from math import cos

# def eqn(x):
#   return x + cos(x)

# myroot = root(eqn, 0)

# print(myroot.x)
# print(myroot)